In [1]:
import pandas as pd
df = pd.read_csv('../2. Datasets/uci_online_retail_cleaned.csv')
df.head()

C:\Users\bryan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,OrderDetails
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2,85123A17850.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,7105317850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1,84406B17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029G17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029E17850.0


In [2]:
df2 = pd.read_csv('../2. Datasets/uci_online_retail_cleaned.csv')
df2.head()

C:\Users\bryan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,OrderDetails
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2,85123A17850.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,7105317850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1,84406B17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029G17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029E17850.0


## Segmenting customers with RFM
Before we dive into Customer Lifetime Value, let’s take a look at the foundational elements of analyzing customer value: **recency, frequency, and monetary value (RFM).**


- **Recency:** the last time that a customer made a purchase.

A customer who has made a purchase recently is more likely to make a repeat purchase than a customer who hasn’t made a purchase in a long time.


- **Frequency:** how many times a customer has made a purchase within a given time frame.

A customer who makes purchases often is more likely to continue to come back than a customer who rarely makes purchases.


- **Monetary value:** the amount of money a customer has spent within that same time frame.

A customer who makes larger purchases is more likely to return than a customer who spends less.

https://www.shopify.com/blog/customer-lifetime-value
https://www.investopedia.com/terms/r/rfm-recency-frequency-monetary-value.asp

In [3]:
def recency(df, date_of_order_col, customer_id_col):
    name = 'RECENCY'
    if name not in df.columns:
        temp1 = df.copy()
        from pandas.tseries.offsets import MonthEnd
        from datetime import datetime
        
        # in temp1 (a copy of the dataframe), change the date to year-month using datetime intelligence.
        # then change to year-month-endofmonth using string concatenation. 
        # then add '23:59:59' at the end using string concatenation
        # then convert this whole thing back into a date using datetime intelligence
        temp1[name] =  pd.to_datetime((pd.to_datetime(max(temp1[date_of_order_col]), format="%Y %m") + MonthEnd(0)).strftime("%Y-%m-%d") + ' 23:59:59')-pd.to_datetime(temp1[date_of_order_col])
        
        # group recency by customer, take the min date
        temp2 = temp1[[customer_id_col, name]].groupby(by=customer_id_col, as_index=False).min()
        
        # left join to original dataframe
        df = pd.merge(df, temp2, left_on=customer_id_col, right_on=customer_id_col, how='left')
        import numpy as np
        df[name] = df[name] / np.timedelta64(1, 'D')
        return df
    else:
        print('already calculated recency (in ' + name + ' column)')
        return df

In [4]:
def frequency(df, date_of_order_col, customer_id_col):
    name = 'FREQUENCY'
    if name not in df.columns:
        temp1 = df.copy()
        temp1[name] = temp1[date_of_order_col]
        # since 1 transaction in 1 line, just group by customers
        temp2 = temp1[[customer_id_col, name]].groupby(by=customer_id_col, as_index=False).count()
        
        # left join to original dataframe
        df = pd.merge(df, temp2, left_on=customer_id_col, right_on=customer_id_col, how='left')
        return df
    else:
        print('already calculated frequency (in ' + name + ' column)')
        return df

In [5]:
def money(df, price_of_product_col, quantity_col, customer_id_col):
    name = 'MONEY'
    if name not in df.columns:
        temp1 = df[[price_of_product_col, quantity_col, customer_id_col]].copy()
        # money spent per transaction is equal the product price times quantity purchased
        temp1[name] = temp1[price_of_product_col]*temp1[quantity_col]
        # group by customers to aggregate money spent
        temp2 = temp1[[customer_id_col, name]].groupby(by=customer_id_col, as_index=False).sum()
        
        # left join to original dataframe
        df = pd.merge(df, temp2, left_on=customer_id_col, right_on=customer_id_col, how='left')
        return df
    else:
        print('already calculated money (in ' + name + ' column)')
        return df

## ==== Recency ====

In [6]:
df = recency(df, 'InvoiceDate', 'CustomerID')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,OrderDetails,RECENCY
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2,85123A17850.0,394.356238
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,7105317850.0,394.356238
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1,84406B17850.0,394.356238
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029G17850.0,394.356238
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029E17850.0,394.356238
...,...,...,...,...,...,...,...,...,...,...,...
394266,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1,NaN,23.489572
394267,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,5,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1,NaN,23.489572
394268,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1,NaN,23.489572
394269,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,11,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1,NaN,23.489572


In [7]:
df2 = recency(df2, 'order_purchase_timestamp', 'customer_id')
df2

,product_id,seller_id,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,...,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,RECENCY
0,00066f42aeeb9f3007548bb9d3f33c38,5670f4db5b62c43d542e1b2d56b0cf7c,f30149f4a8882a08895b6a242aa0d612,86c180c33f454b35e1596a99da3dddc4,delivered,2018-05-20 18:45:21,2018-05-20 18:58:59,2018-05-21 16:09:00,2018-06-06 22:11:56,2018-06-20,...,perfumaria,53,596,6,300.0,20.0,16.0,16.0,perfumery,103.218495
1,00088930e925c41fd95ebfe695fd2655,7142540dd4c91e2237acb7e911c4eba2,f5eda0ded77c1293b04c953138c8331d,68f2b37558e27791155db34bcded5ac0,delivered,2017-12-12 19:20:28,2017-12-12 19:32:19,2017-12-20 20:12:42,2017-12-23 17:11:51,2018-01-05,...,automotivo,56,752,4,1225.0,55.0,10.0,26.0,auto,262.194109
2,0009406fd7479715e4bef61dd91f2462,4a3ca9315b744ce9f8e9374361493884,0bf736fd0fd5169d60de3699fcbcf986,6cd217b674e22cf568f6a2cf6060fd07,delivered,2017-12-21 16:21:47,2017-12-22 17:31:27,2018-01-02 22:27:47,2018-01-06 15:03:41,2018-01-16,...,cama_mesa_banho,50,266,2,300.0,45.0,15.0,35.0,bed_bath_table,253.318194
3,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,3aba44d8e554ab4bb8c09f6f78032ca8,82b838f513e00463174cc7cae7e76c1f,delivered,2018-08-10 13:24:35,2018-08-10 13:35:21,2018-08-13 14:43:00,2018-08-17 21:33:40,2018-08-27,...,utilidades_domesticas,25,364,3,550.0,19.0,24.0,12.0,housewares,21.441250
4,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,6f0dfb5b5398b271cc6bbd9ee263530e,8517e7c86998bf39a540087da6f115d9,delivered,2018-08-01 22:00:33,2018-08-01 22:15:19,2018-08-02 14:20:00,2018-08-07 17:38:52,2018-08-24,...,utilidades_domesticas,25,364,3,550.0,19.0,24.0,12.0,housewares,30.082940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107818,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,2878b4e373a9556df9aa1708a4e5c3c8,a050cd2edce49d430222e6019fa50fd2,delivered,2018-08-07 16:07:57,2018-08-07 16:55:38,2018-08-09 07:06:00,2018-08-09 18:54:34,2018-08-10,...,informatica_acessorios,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,24.327801
107819,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,dd38c566a34ff9deb93ff05d7bd6d0aa,35e8f2cff4eeed925cfa42f6731a4425,delivered,2018-05-01 17:31:41,2018-05-01 17:53:01,2018-05-02 15:04:00,2018-05-03 18:32:48,2018-05-15,...,informatica_acessorios,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,122.269653
107820,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,a8159270bc9cf7d54450e079ba6b7232,839b3007f7e5aee8f8eb130c8fa51c29,delivered,2018-04-01 11:58:04,2018-04-03 05:35:27,2018-04-03 19:47:10,2018-04-20 20:56:36,2018-04-20,...,informatica_acessorios,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,152.501331
107821,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,beaf95465b542dea76df78b2fb86a97a,d14930bf3da4e0ba5a02cc8d6ac80f6e,delivered,2018-04-19 10:12:12,2018-04-19 10:34:35,2018-04-19 19:26:57,2018-04-25 17:58:54,2018-05-15,...,informatica_acessorios,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,134.574850


## ==== Frequency ====

In [8]:
df = frequency(df, 'InvoiceDate', 'CustomerID')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,OrderDetails,RECENCY,FREQUENCY
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2,85123A17850.0,394.356238,297
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,7105317850.0,394.356238,297
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1,84406B17850.0,394.356238,297
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029G17850.0,394.356238,297
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029E17850.0,394.356238,297
...,...,...,...,...,...,...,...,...,...,...,...,...
394266,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1,NaN,23.489572,470
394267,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,5,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1,NaN,23.489572,470
394268,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1,NaN,23.489572,470
394269,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,11,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1,NaN,23.489572,470


In [9]:
df2 = frequency(df2, 'order_purchase_timestamp', 'customer_id')
df2

,product_id,seller_id,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,RECENCY,FREQUENCY
0,00066f42aeeb9f3007548bb9d3f33c38,5670f4db5b62c43d542e1b2d56b0cf7c,f30149f4a8882a08895b6a242aa0d612,86c180c33f454b35e1596a99da3dddc4,delivered,2018-05-20 18:45:21,2018-05-20 18:58:59,2018-05-21 16:09:00,2018-06-06 22:11:56,2018-06-20,...,53,596,6,300.0,20.0,16.0,16.0,perfumery,103.218495,1
1,00088930e925c41fd95ebfe695fd2655,7142540dd4c91e2237acb7e911c4eba2,f5eda0ded77c1293b04c953138c8331d,68f2b37558e27791155db34bcded5ac0,delivered,2017-12-12 19:20:28,2017-12-12 19:32:19,2017-12-20 20:12:42,2017-12-23 17:11:51,2018-01-05,...,56,752,4,1225.0,55.0,10.0,26.0,auto,262.194109,1
2,0009406fd7479715e4bef61dd91f2462,4a3ca9315b744ce9f8e9374361493884,0bf736fd0fd5169d60de3699fcbcf986,6cd217b674e22cf568f6a2cf6060fd07,delivered,2017-12-21 16:21:47,2017-12-22 17:31:27,2018-01-02 22:27:47,2018-01-06 15:03:41,2018-01-16,...,50,266,2,300.0,45.0,15.0,35.0,bed_bath_table,253.318194,1
3,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,3aba44d8e554ab4bb8c09f6f78032ca8,82b838f513e00463174cc7cae7e76c1f,delivered,2018-08-10 13:24:35,2018-08-10 13:35:21,2018-08-13 14:43:00,2018-08-17 21:33:40,2018-08-27,...,25,364,3,550.0,19.0,24.0,12.0,housewares,21.441250,1
4,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,6f0dfb5b5398b271cc6bbd9ee263530e,8517e7c86998bf39a540087da6f115d9,delivered,2018-08-01 22:00:33,2018-08-01 22:15:19,2018-08-02 14:20:00,2018-08-07 17:38:52,2018-08-24,...,25,364,3,550.0,19.0,24.0,12.0,housewares,30.082940,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107818,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,2878b4e373a9556df9aa1708a4e5c3c8,a050cd2edce49d430222e6019fa50fd2,delivered,2018-08-07 16:07:57,2018-08-07 16:55:38,2018-08-09 07:06:00,2018-08-09 18:54:34,2018-08-10,...,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,24.327801,1
107819,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,dd38c566a34ff9deb93ff05d7bd6d0aa,35e8f2cff4eeed925cfa42f6731a4425,delivered,2018-05-01 17:31:41,2018-05-01 17:53:01,2018-05-02 15:04:00,2018-05-03 18:32:48,2018-05-15,...,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,122.269653,1
107820,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,a8159270bc9cf7d54450e079ba6b7232,839b3007f7e5aee8f8eb130c8fa51c29,delivered,2018-04-01 11:58:04,2018-04-03 05:35:27,2018-04-03 19:47:10,2018-04-20 20:56:36,2018-04-20,...,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,152.501331,1
107821,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,beaf95465b542dea76df78b2fb86a97a,d14930bf3da4e0ba5a02cc8d6ac80f6e,delivered,2018-04-19 10:12:12,2018-04-19 10:34:35,2018-04-19 19:26:57,2018-04-25 17:58:54,2018-05-15,...,47,501,5,150.0,16.0,4.0,13.0,computers_accessories,134.574850,1


## ==== Money ====


In [10]:
df = money(df, 'UnitPrice', 'Quantity', 'CustomerID')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,OrderDetails,RECENCY,FREQUENCY,MONEY
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2,85123A17850.0,394.356238,297,5391.21
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,7105317850.0,394.356238,297,5391.21
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1,84406B17850.0,394.356238,297,5391.21
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029G17850.0,394.356238,297,5391.21
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029E17850.0,394.356238,297,5391.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
394266,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1,NaN,23.489572,470,6237.05
394267,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,5,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1,NaN,23.489572,470,6237.05
394268,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1,NaN,23.489572,470,6237.05
394269,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,11,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1,NaN,23.489572,470,6237.05


In [11]:
df2['qty'] = 1
df2 = money(df2, 'price', 'qty', 'customer_id')
df2 = df2.drop(['qty'], axis=1)
df2

,product_id,seller_id,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,RECENCY,FREQUENCY,MONEY
0,00066f42aeeb9f3007548bb9d3f33c38,5670f4db5b62c43d542e1b2d56b0cf7c,f30149f4a8882a08895b6a242aa0d612,86c180c33f454b35e1596a99da3dddc4,delivered,2018-05-20 18:45:21,2018-05-20 18:58:59,2018-05-21 16:09:00,2018-06-06 22:11:56,2018-06-20,...,596,6,300.0,20.0,16.0,16.0,perfumery,103.218495,1,101.65
1,00088930e925c41fd95ebfe695fd2655,7142540dd4c91e2237acb7e911c4eba2,f5eda0ded77c1293b04c953138c8331d,68f2b37558e27791155db34bcded5ac0,delivered,2017-12-12 19:20:28,2017-12-12 19:32:19,2017-12-20 20:12:42,2017-12-23 17:11:51,2018-01-05,...,752,4,1225.0,55.0,10.0,26.0,auto,262.194109,1,129.90
2,0009406fd7479715e4bef61dd91f2462,4a3ca9315b744ce9f8e9374361493884,0bf736fd0fd5169d60de3699fcbcf986,6cd217b674e22cf568f6a2cf6060fd07,delivered,2017-12-21 16:21:47,2017-12-22 17:31:27,2018-01-02 22:27:47,2018-01-06 15:03:41,2018-01-16,...,266,2,300.0,45.0,15.0,35.0,bed_bath_table,253.318194,1,229.00
3,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,3aba44d8e554ab4bb8c09f6f78032ca8,82b838f513e00463174cc7cae7e76c1f,delivered,2018-08-10 13:24:35,2018-08-10 13:35:21,2018-08-13 14:43:00,2018-08-17 21:33:40,2018-08-27,...,364,3,550.0,19.0,24.0,12.0,housewares,21.441250,1,58.90
4,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,6f0dfb5b5398b271cc6bbd9ee263530e,8517e7c86998bf39a540087da6f115d9,delivered,2018-08-01 22:00:33,2018-08-01 22:15:19,2018-08-02 14:20:00,2018-08-07 17:38:52,2018-08-24,...,364,3,550.0,19.0,24.0,12.0,housewares,30.082940,1,58.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107818,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,2878b4e373a9556df9aa1708a4e5c3c8,a050cd2edce49d430222e6019fa50fd2,delivered,2018-08-07 16:07:57,2018-08-07 16:55:38,2018-08-09 07:06:00,2018-08-09 18:54:34,2018-08-10,...,501,5,150.0,16.0,4.0,13.0,computers_accessories,24.327801,1,34.99
107819,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,dd38c566a34ff9deb93ff05d7bd6d0aa,35e8f2cff4eeed925cfa42f6731a4425,delivered,2018-05-01 17:31:41,2018-05-01 17:53:01,2018-05-02 15:04:00,2018-05-03 18:32:48,2018-05-15,...,501,5,150.0,16.0,4.0,13.0,computers_accessories,122.269653,1,34.99
107820,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,a8159270bc9cf7d54450e079ba6b7232,839b3007f7e5aee8f8eb130c8fa51c29,delivered,2018-04-01 11:58:04,2018-04-03 05:35:27,2018-04-03 19:47:10,2018-04-20 20:56:36,2018-04-20,...,501,5,150.0,16.0,4.0,13.0,computers_accessories,152.501331,1,29.99
107821,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,beaf95465b542dea76df78b2fb86a97a,d14930bf3da4e0ba5a02cc8d6ac80f6e,delivered,2018-04-19 10:12:12,2018-04-19 10:34:35,2018-04-19 19:26:57,2018-04-25 17:58:54,2018-05-15,...,501,5,150.0,16.0,4.0,13.0,computers_accessories,134.574850,1,34.99


## ==== Normalisation ====

To ensure Recency, Frequency, and Money are all on the same scale, we will normalise them to between 0 and 1 based on the range of raw data available to each column.

Additionally, for Recency, we need to invert it because the value refers to days since last purchase. Hence, the higher the number, the lower the Recency.

In [12]:
if (min(df['RECENCY'])>1):
    df['RECENCY'] = 1/df['RECENCY']
if (min(df2['RECENCY'])>1):
    df2['RECENCY'] = 1/df2['RECENCY']

In [13]:
def normalise(df, column):
    name = column+'_normalised'
    df[name] = ( df[column]-min(df[column]) ) / ( max(df[column])-min(df[column]) ) * (5-1) + 1
    return df

In [14]:
df = normalise(normalise(normalise(df, 'RECENCY'), 'FREQUENCY'), 'MONEY')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,OrderDetails,RECENCY,FREQUENCY,MONEY,RECENCY_normalised,FREQUENCY_normalised,MONEY_normalised
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2,85123A17850.0,0.002536,297,5391.21,1.000752,1.160108,1.077756
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,7105317850.0,0.002536,297,5391.21,1.000752,1.160108,1.077756
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1,84406B17850.0,0.002536,297,5391.21,1.000752,1.160108,1.077756
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029G17850.0,0.002536,297,5391.21,1.000752,1.160108,1.077756
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,84029E17850.0,0.002536,297,5391.21,1.000752,1.160108,1.077756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394266,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1,NaN,0.042572,470,6237.05,4.815071,1.253685,1.089962
394267,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,5,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1,NaN,0.042572,470,6237.05,4.815071,1.253685,1.089962
394268,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1,NaN,0.042572,470,6237.05,4.815071,1.253685,1.089962
394269,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,11,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1,NaN,0.042572,470,6237.05,4.815071,1.253685,1.089962


In [15]:
df2 = normalise(normalise(normalise(df2, 'RECENCY'), 'FREQUENCY'), 'MONEY')
df2

,product_id,seller_id,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,...,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,RECENCY,FREQUENCY,MONEY,RECENCY_normalised,FREQUENCY_normalised,MONEY_normalised
0,00066f42aeeb9f3007548bb9d3f33c38,5670f4db5b62c43d542e1b2d56b0cf7c,f30149f4a8882a08895b6a242aa0d612,86c180c33f454b35e1596a99da3dddc4,delivered,2018-05-20 18:45:21,2018-05-20 18:58:59,2018-05-21 16:09:00,2018-06-06 22:11:56,2018-06-20,...,20.0,16.0,16.0,perfumery,0.009688,1,101.65,1.079008,1.0,1.030002
1,00088930e925c41fd95ebfe695fd2655,7142540dd4c91e2237acb7e911c4eba2,f5eda0ded77c1293b04c953138c8331d,68f2b37558e27791155db34bcded5ac0,delivered,2017-12-12 19:20:28,2017-12-12 19:32:19,2017-12-20 20:12:42,2017-12-23 17:11:51,2018-01-05,...,55.0,10.0,26.0,auto,0.003814,1,129.90,1.023027,1.0,1.038410
2,0009406fd7479715e4bef61dd91f2462,4a3ca9315b744ce9f8e9374361493884,0bf736fd0fd5169d60de3699fcbcf986,6cd217b674e22cf568f6a2cf6060fd07,delivered,2017-12-21 16:21:47,2017-12-22 17:31:27,2018-01-02 22:27:47,2018-01-06 15:03:41,2018-01-16,...,45.0,15.0,35.0,bed_bath_table,0.003948,1,229.00,1.024301,1.0,1.067906
3,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,3aba44d8e554ab4bb8c09f6f78032ca8,82b838f513e00463174cc7cae7e76c1f,delivered,2018-08-10 13:24:35,2018-08-10 13:35:21,2018-08-13 14:43:00,2018-08-17 21:33:40,2018-08-27,...,19.0,24.0,12.0,housewares,0.046639,1,58.90,1.431145,1.0,1.017278
4,000b8f95fcb9e0096488278317764d19,40ec8ab6cdafbcc4f544da38c67da39a,6f0dfb5b5398b271cc6bbd9ee263530e,8517e7c86998bf39a540087da6f115d9,delivered,2018-08-01 22:00:33,2018-08-01 22:15:19,2018-08-02 14:20:00,2018-08-07 17:38:52,2018-08-24,...,19.0,24.0,12.0,housewares,0.033241,1,58.90,1.303467,1.0,1.017278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107818,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,2878b4e373a9556df9aa1708a4e5c3c8,a050cd2edce49d430222e6019fa50fd2,delivered,2018-08-07 16:07:57,2018-08-07 16:55:38,2018-08-09 07:06:00,2018-08-09 18:54:34,2018-08-10,...,16.0,4.0,13.0,computers_accessories,0.041105,1,34.99,1.378408,1.0,1.010161
107819,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,dd38c566a34ff9deb93ff05d7bd6d0aa,35e8f2cff4eeed925cfa42f6731a4425,delivered,2018-05-01 17:31:41,2018-05-01 17:53:01,2018-05-02 15:04:00,2018-05-03 18:32:48,2018-05-15,...,16.0,4.0,13.0,computers_accessories,0.008179,1,34.99,1.064622,1.0,1.010161
107820,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,a8159270bc9cf7d54450e079ba6b7232,839b3007f7e5aee8f8eb130c8fa51c29,delivered,2018-04-01 11:58:04,2018-04-03 05:35:27,2018-04-03 19:47:10,2018-04-20 20:56:36,2018-04-20,...,16.0,4.0,13.0,computers_accessories,0.006557,1,29.99,1.049171,1.0,1.008673
107821,fffdb2d0ec8d6a61f0a0a0db3f25b441,9f505651f4a6abe901a56cdc21508025,beaf95465b542dea76df78b2fb86a97a,d14930bf3da4e0ba5a02cc8d6ac80f6e,delivered,2018-04-19 10:12:12,2018-04-19 10:34:35,2018-04-19 19:26:57,2018-04-25 17:58:54,2018-05-15,...,16.0,4.0,13.0,computers_accessories,0.007431,1,34.99,1.057495,1.0,1.010161


## Customer lifetime value formula
To calculate the customer lifetime value for each of your customer segments, you’ll need to track down three key pieces of data within your pre-established timeframe: **average order value, purchase frequency, and customer value.**

- **Average order value:** average amount of money a customer spends every time they place an order, i.e. take total revenue and divide it by the total number of orders.

- **Purchase frequency:** average amount of orders placed by each customer, i.e. divide total number of orders by total number of unique customers. The result will be your purchase frequency.

- **Customer lifetime value:** average monetary value each customer brings to your business during a time frame, i.e. multiply your average order value by your purchase frequency.

https://stacktome.com/blog/diving-deeper-into-customer-segmentation-rfm-and-clv
https://www.shopify.com/blog/customer-lifetime-value

In [16]:
def getOrdVal(df):
    temp = df.copy()
    temp['total'] = df['UnitPrice']*df['Quantity']
    totalRev = sum(temp['total'])
    totalOrd = len(df['InvoiceDate'])
    return totalRev/totalOrd

In [17]:
getOrdVal(df)

21.221299406948052

In [18]:
def getPurFre(df):
    totalOrd = len(df['InvoiceDate'])
    totalCus = len(df['CustomerID'].unique())
    return totalOrd/totalCus

In [19]:
getPurFre(df)

91.22420175844516

In [20]:
def getCLV(df):
    return getOrdVal(df)*getPurFre(df)

In [21]:
getCLV(df)

1935.8960986758018

In [22]:
df.to_csv('Data/uci_online_retail_cleaned_CLV.csv', index=False)

In [23]:
df2.to_csv('Data/Orders_merged_CLV.csv', index=False)